# T1547.009 - Shortcut Modification
Adversaries may create or modify shortcuts that can execute a program during system boot or user login. Shortcuts or symbolic links are used to reference other files or programs that will be opened or executed when the shortcut is clicked or executed by a system startup process.

Adversaries may abuse shortcuts in the startup folder to execute their tools and achieve persistence.(Citation: Shortcut for Persistence ) Although often used as payloads in an infection chain (e.g. [Spearphishing Attachment](https://attack.mitre.org/techniques/T1566/001)), adversaries may also create a new shortcut as a means of indirection, while also abusing [Masquerading](https://attack.mitre.org/techniques/T1036) to make the malicious shortcut appear as a legitimate program. Adversaries can also edit the target path or entirely replace an existing shortcut so their malware will be executed instead of the intended legitimate program.

Shortcuts can also be abused to establish persistence by implementing other methods. For example, LNK browser extensions may be modified (e.g. [Browser Extensions](https://attack.mitre.org/techniques/T1176)) to persistently launch malware.

## Atomic Tests

### Atomic Test #1 - Shortcut ModificationThis test to simulate shortcut modification and then execute. example shortcut (*.lnk , .url) strings check with powershell;
gci -path "C:\Users" -recurse -include *.url -ea SilentlyContinue | Select-String -Pattern "exe" | FL.
Upon execution, calc.exe will be launched.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
echo [InternetShortcut] > %temp%\T1547.009_modified_shortcut.url
echo URL=C:\windows\system32\calc.exe >> %temp%\T1547.009_modified_shortcut.url
%temp%\T1547.009_modified_shortcut.url
```

In [ ]:
Invoke-AtomicTest T1547.009 -TestNumbers 1

#### Cleanup: 
```cmd
del -f %temp%\T1547.009_modified_shortcut.url >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1547.009 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Create shortcut to cmd in startup foldersLNK file to launch CMD placed in startup folder. Upon execution, open File Explorer and browse to "%APPDATA%\Microsoft\Windows\Start Menu\Programs\Startup\"
to view the new shortcut.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$Shell = New-Object -ComObject ("WScript.Shell")
$ShortCut = $Shell.CreateShortcut("$env:APPDATA\Microsoft\Windows\Start Menu\Programs\Startup\T1547.009.lnk")
$ShortCut.TargetPath="cmd.exe"
$ShortCut.WorkingDirectory = "C:\Windows\System32";
$ShortCut.WindowStyle = 1;
$ShortCut.Description = "T1547.009.";
$ShortCut.Save()

$Shell = New-Object -ComObject ("WScript.Shell")
$ShortCut = $Shell.CreateShortcut("$env:ProgramData\Microsoft\Windows\Start Menu\Programs\Startup\T1547.009.lnk")
$ShortCut.TargetPath="cmd.exe"
$ShortCut.WorkingDirectory = "C:\Windows\System32";
$ShortCut.WindowStyle = 1;
$ShortCut.Description = "T1547.009.";
$ShortCut.Save()
```

In [ ]:
Invoke-AtomicTest T1547.009 -TestNumbers 2

#### Cleanup: 
```powershell
Remove-Item "$env:APPDATA\Microsoft\Windows\Start Menu\Programs\Startup\T1547.009.lnk" -ErrorAction Ignore
Remove-Item "$env:ProgramData\Microsoft\Windows\Start Menu\Programs\Startup\T1547.009.lnk" -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1547.009 -TestNumbers 2 -Cleanup

## Detection
Since a shortcut's target path likely will not change, modifications to shortcut files that do not correlate with known software changes, patches, removal, etc., may be suspicious. Analysis should attempt to relate shortcut file change or creation events to other potentially suspicious events based on known adversary behavior such as process launches of unknown executables that make network connections.

Monitor for LNK files created with a Zone Identifier value greater than 1, which may indicate that the LNK file originated from outside of the network.(Citation: BSidesSLC 2020 - LNK Elastic)